In [24]:
# upgrade python
# !pip install --upgrade pip
# install collapsible heading plugin
# !pip install jupyter_contrib_nbextensions

In [22]:
import requests as req
import json

import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 200px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)
        
import socket

def get_host_ip():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    finally:
        s.close()

    return ip

theIP = get_host_ip()
print ("Local host IP: " + theIP)

Local host IP: 172.16.1.148


### KeyCloak OpenID Connect URIs

In [30]:
urlKeys = ['issuer', 
            'authorization_endpoint', 
            'token_endpoint', 
            'token_introspection_endpoint', 
            'userinfo_endpoint', 
            'end_session_endpoint', 
            'jwks_uri']
realm = "Service" 
kcHost = "http://{}:8080".format(theIP)
kcDiscoveryUrl = "{}/auth/realms/{}/.well-known/openid-configuration".format(kcHost, realm)
print("discovery: " + kcDiscoveryUrl)
resp  = req.get(kcDiscoveryUrl)
if resp.status_code != 200:
    print(resp.text)
else:
    oidcConf = json.loads(resp.text)
    for urlKey in urlKeys:
        print(urlKey + ": " + oidcConf[urlKey])
print('user account: {}/auth/realms/{}/account/'.format(kcHost, realm))
authUrl  = oidcConf['authorization_endpoint']
tokenUrl = oidcConf['token_endpoint']
tokenIntroUrl = oidcConf['token_introspection_endpoint']
userUrl  = oidcConf['userinfo_endpoint']
kcIssuer = oidcConf['issuer']

discovery: http://172.16.1.148:8080/auth/realms/Service/.well-known/openid-configuration
issuer: http://172.16.1.148:8080/auth/realms/Service
authorization_endpoint: http://172.16.1.148:8080/auth/realms/Service/protocol/openid-connect/auth
token_endpoint: http://172.16.1.148:8080/auth/realms/Service/protocol/openid-connect/token
token_introspection_endpoint: http://172.16.1.148:8080/auth/realms/Service/protocol/openid-connect/token/introspect
userinfo_endpoint: http://172.16.1.148:8080/auth/realms/Service/protocol/openid-connect/userinfo
end_session_endpoint: http://172.16.1.148:8080/auth/realms/Service/protocol/openid-connect/logout
jwks_uri: http://172.16.1.148:8080/auth/realms/Service/protocol/openid-connect/certs
user account: http://172.16.1.148:8080/auth/realms/Service/account/


In [31]:
client_id = "Harmoney Service"
client_secret = "932a1892-6ead-4fe9-b0b8-a75f6f8e4b19"

### Get Access Token from Keycloak

In [11]:
%%time
reqJson = {
    "client_id": client_id,
    "client_secret": client_secret,
    "username": "service",
    "password": "service",
    "grant_type": "password",
    "scope": "openid" 
}

header = {
    "Content-Type": "application/x-www-form-urlencoded",
} 

response = req.post(tokenUrl, data=reqJson, headers=header)
print(response.status_code)
if response.status_code > 200:
    print (response.text)
else:
    accessToken = json.loads(response.text)['access_token']

200
Wall time: 125 ms


### Set up kong-oidc plugins

In [20]:
kongAdminUrl = "http://{}:8001".format(theIP)
resp = req.get(kongAdminUrl)
print(resp.status_code)
RenderJSON(json.loads(resp.text))

200


In [32]:
oidcJson = {
    "name": "oidc",
    "config.client_id": client_id,
    "config.client_secret": client_secret,
    "config.discovery": kcDiscoveryUrl,
    "config.introspection_endpoint": tokenIntroUrl,
    "config.token_endpoint": tokenUrl,
    "config.username": "service",
    "config.password": "service",
}

header = {
    "Content-Type": "application/x-www-form-urlencoded",
} 

# Add the plugin globally
#response = req.post(kongAdminUrl+"/plugins", data=oidcJson, headers=header)

# Add the plugin for a Service
#response = req.post(kongAdminUrl+"/services/8b20527c-bcbf-41e8-bbcf-08d0d1a6560e/plugins", data=oidcJson, headers=header)

# Add the plugin for a Route
response = req.post(kongAdminUrl+"/routes/42a0488c-e642-4de0-ae89-3e51dd1505c8/plugins", data=oidcJson, headers=header)
print(response.status_code)
if response.status_code >= 300:
    print (response.text)

201


### Access the service with API Key

In [33]:
b2bUrl = "http://{}:8000/proviso".format(theIP)
header = {
    "apikey": "QLc0Ui1P7pr5LAxFYciBxDLY3wmlJYA",
}
response = req.get(b2bUrl, headers=header)  
print("{} {}".format(response.status_code, response.text))

200 Welcome to Harmoney through Kong API Key!
